测试kafka的接收
控制台上可以这样接收：
kafka-console-consumer.sh --bootstrap-server localhost:9092 --topic user_list_click --from-beginning


In [ ]:
import time

import kafka
from kafka import KafkaConsumer
consumer = KafkaConsumer('input', bootstrap_servers=['localhost:9092'])
for msg,i in zip(consumer,range(1,1000)):
    recv = "%s:%d:%d: key=%s value=%s" % (msg.topic, msg.partition, msg.offset, msg.key, msg.value)
    print(recv)
    if i %200==0:
        break
print("退出kafka连接")
consumer.close()


In [ ]:
from kafka import KafkaConsumer
consumer = KafkaConsumer('testtest', bootstrap_servers=['localhost:9092'])
for msg in consumer:
    recv = "%s:%d:%d: key=%s value=%s" % (msg.topic, msg.partition, msg.offset, msg.key, msg.value)
    print(recv)
consumer.close()

尝试用spark连接kafka
注意需要将这两个jar包放在kafka的线上jars目录下：
spark-sql-kafka-0-10_2.12-2.4.5.jar
spark-streaming-kafka-0-10-assembly_2.12-2.4.5.jar
下载地址：
https://search.maven.org/artifact/org.apache.spark/spark-streaming-kafka-0-10-assembly_2.12/2.4.5/jar
https://search.maven.org/artifact/org.apache.spark/spark-sql-kafka-0-10_2.12/2.4.5/jar



In [ ]:
import time
from pyspark.sql import SparkSession
from pyspark import SparkContext
# 如果启动有问题，这两个环境可以强行指定，正常包不冲突时不用指定就行
run_env ={
    "PYSPARK_SUBMIT_ARGS":'--packages /zzti/libs/spark/jars/spark-sql-kafka-0-10_2.12-2.4.5.jar pyspark-shell',
    "SPARK_KAFKA_VERSION":0.10
}
sc = SparkContext(master="yarn",
                  #environment=run_env,
                  appName="test_kafka")
ss = SparkSession(sc)
ss

In [ ]:
print(time.time())
stream = ss\
    .readStream\
    .format("kafka")\
    .option("kafka.bootstrap.servers", "localhost:9092")\
    .option("subscribe", "input")\
    .load()
stream

In [ ]:
# 打印输入数据结构
stream.printSchema()

In [ ]:
print(time.time())
result = stream.selectExpr("CAST(value AS STRING)")\
    .writeStream\
    .outputMode('append')\
    .format('console')\
    .start()
result.awaitTermination()

In [ ]:
sc.stop()

In [8]:
print("end")






end
